In [1]:
!git clone https://github.com/indra622/sgspeech_private.git -b tiny_sgspeech

Cloning into 'sgspeech_private'...
remote: Enumerating objects: 254, done.
remote: Counting objects: 100% (254/254), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 254 (delta 102), reused 213 (delta 61), pack-reused 0
Receiving objects: 100% (254/254), 170.65 KiB | 14.22 MiB/s, done.
Resolving deltas: 100% (102/102), done.


In [ ]:
!wget https://www.openslr.org/resources/12/train-clean-100.tar.gz
!wget https://www.openslr.org/resources/12/test-clean.tar.gz
!wget https://www.openslr.org/resources/12/test-other.tar.gz
!wget https://www.openslr.org/resources/12/dev-clean.tar.gz
!wget https://www.openslr.org/resources/12/dev-other.tar.gz

--2021-06-10 06:39:12--  https://www.openslr.org/resources/12/train-clean-100.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6387309499 (5.9G) [application/x-gzip]
Saving to: ‘train-clean-100.tar.gz.1’

train-clean-100.tar  28%[====>               ]   1.71G  3.28MB/s    eta 8m 33s ^C
--2021-06-10 06:42:40--  https://www.openslr.org/resources/12/test-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346663984 (331M) [application/x-gzip]
Saving to: ‘test-clean.tar.gz.1’

test-clean.tar.gz.1  67%[============>       ] 222.02M  14.7MB/s    eta 7s     ^C
--2021-06-10 06:42:56--  https://www.openslr.org/resources/12/test-other.tar.gz
Resolving www.openslr.org (www.openslr.org)... 

In [ ]:
!tar -xvzf test-clean.tar.gz
!tar -xvzf train-clean-100.tar.gz
!tar -xvzf test-other.tar.gz
!tar -xvzf dev-clean.tar.gz
!tar -xvzf dev-other.tar.gz

In [ ]:
!python sgspeech_private/create_librispeech_trans.py --dir /content/LibriSpeech/train-clean-100 /content/LibriSpeech/train-clean-100/transcripts.tsv
!python sgspeech_private/create_librispeech_trans.py --dir /content/LibriSpeech/dev-clean /content/LibriSpeech/dev-clean/transcripts.tsv
!python sgspeech_private/create_librispeech_trans.py --dir /content/LibriSpeech/dev-other /content/LibriSpeech/dev-other/transcripts.tsv
!python sgspeech_private/create_librispeech_trans.py --dir /content/LibriSpeech/test-clean /content/LibriSpeech/test-clean/transcripts.tsv
!python sgspeech_private/create_librispeech_trans.py --dir /content/LibriSpeech/test-other /content/LibriSpeech/test-other/transcripts.tsv

2021-06-10 06:43:18.523338: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
[Loading]: 100% 585/585 [03:00<00:00,  3.25it/s]
[Writing]: 100% 28539/28539 [00:00<00:00, 1364256.64it/s]
2021-06-10 06:46:21.877924: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
[Loading]: 100% 97/97 [00:09<00:00,  9.95it/s]
[Writing]: 100% 2703/2703 [00:00<00:00, 1473512.31it/s]
2021-06-10 06:46:34.396761: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
[Loading]: 100% 91/91 [00:09<00:00,  9.96it/s]
[Writing]: 100% 2864/2864 [00:00<00:00, 1495392.34it/s]
2021-06-10 06:46:46.079037: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
[Loading]: 100% 87/87 [00:09<00:00,  8.71it/s]
[Writing]: 100% 2620/2620 [00:00<00:00, 1210784.10it/s]


In [ ]:
!cd sgspeech_private && pip install .

Processing /content/sgspeech_private
  Created wheel for SGSpeech: filename=SGSpeech-0.0.1-cp37-none-any.whl size=1190 sha256=16d29554a0fc81187eaa7c752ac23031a4382c3c58cf875578ae104ae98a4ff6
  Stored in directory: /tmp/pip-ephem-wheel-cache-_hv9268l/wheels/a7/fa/4f/adc33db7c98d90534d0aaf5f93254c7f765ee7a83431513da7
Successfully built SGSpeech
  Found existing installation: SGSpeech 0.0.1
    Uninstalling SGSpeech-0.0.1:
      Successfully uninstalled SGSpeech-0.0.1


In [ ]:
import os
import math
import argparse
from sgspeech_private.sgspeech.utils import setup_environment, setup_strategy

setup_environment()
import tensorflow as tf

tf.keras.backend.clear_session()

strategy = setup_strategy([0])

from sgspeech_private.sgspeech.configs.config import Config
from sgspeech_private.sgspeech.datasets.speech_dataset import SpeechSliceDataset
from sgspeech_private.sgspeech.featurizers.speech_featurizer import NumpySpeechFeaturizer
from sgspeech_private.sgspeech.featurizers.text_featurizer import CharFeaturizer
from sgspeech_private.sgspeech.runners.transducer_runners import TransducerTrainer
from sgspeech_private.sgspeech.models.conformer import Conformer
from sgspeech_private.sgspeech.optimizers.schedules import TransformerSchedule

config = Config('/content/sgspeech_private/config.yml')
speech_featurizer = NumpySpeechFeaturizer(config.speech_config)
text_featurizer = CharFeaturizer(config.decoder_config)

train_dataset = SpeechSliceDataset(
    speech_featurizer=speech_featurizer, text_featurizer=text_featurizer,
    **vars(config.learning_config.train_dataset_config)
)
eval_dataset = SpeechSliceDataset(
    speech_featurizer=speech_featurizer, text_featurizer=text_featurizer,
    **vars(config.learning_config.eval_dataset_config)
)

conformer_trainer = TransducerTrainer(
    config=config.learning_config.running_config,
    text_featurizer=text_featurizer, strategy=strategy
)

with conformer_trainer.strategy.scope():
    # build model
    conformer = Conformer(**config.model_config, vocabulary_size=text_featurizer.num_classes)
    conformer._build(speech_featurizer.shape)
    conformer.summary(line_length=120)

    optimizer_config = config.learning_config.optimizer_config
    optimizer = tf.keras.optimizers.Adam(
        TransformerSchedule(
            d_model=conformer.dmodel,
            warmup_steps=optimizer_config["warmup_steps"],
            max_lr=(0.05 / math.sqrt(conformer.dmodel))
        ),
        beta_1=optimizer_config["beta1"],
        beta_2=optimizer_config["beta2"],
        epsilon=optimizer_config["epsilon"]
    )

conformer_trainer.compile(model=conformer, optimizer=optimizer,
                          max_to_keep=10)

conformer_trainer.fit(train_dataset, eval_dataset, train_bs=4, eval_bs=1)

Run on 1 Physical GPUs
Use RNNT loss in TensorFlow
Model: "conformer_encoder"
________________________________________________________________________________________________________________________
Layer (type)                                          Output Shape                                    Param #           
conformer_encoder_subsampling (Conv2dSubsampling)     multiple                                        188208            
________________________________________________________________________________________________________________________
conformer_encoder_pe (PositionalEncodingConcat)       multiple                                        0                 
________________________________________________________________________________________________________________________
conformer_encoder_linear (Dense)                      multiple                                        414864            
___________________________________________________________________________

[Train] [Epoch 2/20 |█▏                  | 8096/142680 [1:17:33<317:16:24,  8.49s/batch, transducer_loss=209.93613]